Take all the Latin text, clean it and tokenize it, then write to new file in machine translation directory

In [241]:
from collections import defaultdict
import json
import os
import re
import string

from cltk.stem.latin.j_v import JVReplacer
from cltk.stem.lemma import LemmaReplacer
from cltk.tokenize.word import WordTokenizer
from cltk.tokenize.sentence import TokenizeSentence

In [2]:
tok = TokenizeSentence('latin').tokenize
wt = WordTokenizer('latin')
j = JVReplacer()

In [247]:
os.chdir('latin_text_latin_library/')

In [252]:
for directory in os.listdir()[2:]:
    dname = '/Users/nickybangs/ds/metis/machine_translation/latin_corpus_tokenized/{}'.format(directory)
    os.mkdir(dname)
    for file in os.listdir(directory):
        fname = file.split('.txt')[0]
        fname = dname + '/' + fname + '_tokens.json'
        f = open(directory + '/' + file)
        text = f.read()
        f.close()
        text = re.sub(r'\t', ' ',text).replace('´','')
        text = re.sub(r'  *', ' ', text)
        text = re.sub(r'[0-9]+\.?','',text)
        text = re.sub(r'\[.\]','',text)
        text = j.replace(text.lower())
        text = text.splitlines()

        text = text[1:] + text[:-6]

        while '' in text:
            text.remove('')
        while ' ' in text:
            text.remove(' ')

        sentence_tokens = []

        for line in text:
            line = line.replace('\n', ' ')
            line_tokens = tok(line)
            for token in line_tokens:
                token = re.sub(r'[{}]'.format(string.punctuation), ' ', token).strip()
                token = re.sub(r'  *',' ',token)
                token = wt.tokenize(token)
                if len(token) < 3:
                    continue
                else:
                    sentence_tokens.append(token)
        f = open(fname,'w')
        f.write(json.dumps(sentence_tokens))
        f.close()